In [5]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions, preprocess_input


def VGGCustom(input_tensor=None, classes=2):
    img_rows, img_cols = 300, 300  # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)

    img_input = Input(shape=img_dim)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.
    model = Model(inputs=img_input, outputs=x, name='VGG16')

    return model


# Test the function
model = VGGCustom()
print(model.summary())

Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 300, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 75, 128)       0     

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

# Directory containing your training data
train_data_dir = '/home/darksst/Desktop/SHAP_Project/Data/tiny-imagenet-200/train'

# Data generators with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Using 20% of data for validation

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(300, 300),
    batch_size=1,
    class_mode='categorical',
    subset='training')  # Training data

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(300, 300),
    batch_size=1,
    class_mode='categorical',
    subset='validation')  # Validation data

# Create and compile the model
model = VGGCustom(input_tensor=None, classes=200)  # Adjust the number of classes based on your dataset
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.


In [8]:
# Train the model
history = model.fit(
    train_generator,
    epochs=50,  # Number of epochs to train
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

Epoch 1/50


2024-04-25 01:43:13.712418: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-25 01:43:14.054154: I external/local_xla/xla/service/service.cc:168] XLA service 0x76ded904eac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-25 01:43:14.054174: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-04-25 01:43:14.060360: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714027394.133727   24077 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-04-25 01:43:24.413862: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.27GiB (rounded to 1358954752)requested by op 
2024

ResourceExhaustedError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_26 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/darksst/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/darksst/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/darksst/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/darksst/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/darksst/.local/lib/python3.10/site-packages/ipykernel/ker